In [ ]:
import xarray as xr
import os

filenames = ["1000mb2023.nc", "925mb2023.nc"]

output_filename = "1000+925_23.nc"

local_paths = filenames

try:
    print("Opening files from local disk...")
    print("Merging datasets...")

    with xr.open_mfdataset(
        local_paths,
        engine='netcdf4',
        combine='nested',
        concat_dim='pressure_level',
        chunks={'time': 30}
    ) as ds:    

        print("Saving merged dataset...")
        ds.to_netcdf(output_filename)
        print("Dataset saved successfully!")       

except Exception as e:
    print(f"Error occurred: {e}")   
print("Process completed!")

In [ ]:
import xarray as xr
import pandas as pd
import os

input_file = "/home/monad/gcs-bucket/2023/250-50_23.nc"
output_folder = "/home/monad/gcs-bucket/2023/By_month"
os.makedirs(output_folder, exist_ok=True)

ds = xr.open_dataset(input_file)

if not pd.api.types.is_datetime64_any_dtype(ds['valid_time'].dtype):
    ds['valid_time'] = pd.to_datetime(ds['valid_time'].values)

for month in range(1, 13):
    ds_month = ds.sel(valid_time=ds['valid_time'].dt.month == month)
    
    if ds_month.valid_time.size > 0:
        month_name = pd.Timestamp(year=2023, month=month, day=1).strftime('%B').lower()
        output_file = os.path.join(output_folder, f"{month_name}_250-50_23.nc")
        
        print(f"Saving {output_file} ...")
        ds_month.to_netcdf(output_file)
        
print("✅ All monthly files created successfully.")


In [ ]:
import xarray as xr
import os
import calendar

# ==== SETTINGS ====
input_file = "/home/monad/gcs-bucket/gs_2023_2024_merged.nc" 
output_dir_2023 = "/home/monad/gcs-bucket/2023/By_month"
output_dir_2024 = "/home/monad/gcs-bucket/2024/By_month"

os.makedirs(output_dir_2023, exist_ok=True)
os.makedirs(output_dir_2024, exist_ok=True)

ds = xr.open_dataset(input_file)

lsm_static = ds["lsm"].isel(valid_time=0).drop_vars("valid_time")
ds["lsm"] = lsm_static

# ==== RENAME VARIABLES ====
rename_map = {
    "u10": "10m_u_component_of_wind",
    "v10": "10m_v_component_of_wind",
    "t2m": "2m_temperature",
    "msl": "mean_sea_level_pressure",
    "lsm": "land_sea_mask"
}
ds = ds.rename(rename_map)

# ==== SPLIT BY YEAR & MONTH ====
for year in [2023, 2024]:
    for month in range(1, 13):
        subset = ds.sel(valid_time=f"{year}-{month:02d}")

        month_name = calendar.month_name[month].lower()

        if year == 2023:
            file_path = os.path.join(output_dir_2023, f"{month_name}_single_23.nc")
        else:
            file_path = os.path.join(output_dir_2024, f"{month_name}_single_24.nc")

        subset.to_netcdf(file_path)
        print(f"Saved {file_path}")

print("✅ Done! 24 monthly files created with month names.")


In [ ]:
import xarray as xr

input_file = "/home/monad/gcs-bucket/2024/By_month/geopotential_at_surface.nc"
output_file = "/home/monad/gcs-bucket/2024/By_month/geo_at_sur.nc"

# Open the file
ds = xr.open_dataset(input_file)

# Select first (and only) time step and drop the time dimension
ds_no_time = ds.isel(valid_time=0).drop_vars("valid_time")

# Rename variable
ds_no_time = ds_no_time.rename({"z": "geopotential_at_surface"})

# Remove extra coordinates if you want a cleaner file
ds_no_time = ds_no_time.drop_vars(["number", "expver"])

# Save as new file
ds_no_time.to_netcdf(output_file)

print(f"Saved modified dataset as: {output_file}")


In [ ]:
import xarray as xr
import glob
import os

folder_path = "/home/monad/gcs-bucket/2023/By_month/"

output_file = "/home/monad/gcs-bucket/Input files/january_23.nc"

january_files = glob.glob(os.path.join(folder_path, "*january*.nc"))


geo_file = os.path.join(folder_path, "geo_at_sur.nc")
if os.path.exists(geo_file):
    january_files.append(geo_file)


datasets = []
for file in january_files:
    print(f"Opening {file}")
    ds = xr.open_dataset(file)

    if 'expver' in ds.coords:
        ds = ds.drop_vars('expver')
    
    datasets.append(ds)


merged_ds = xr.merge(datasets, compat="override") 

for ds in datasets:
    ds.close()

# Rename variables and coordinates
rename_dict = {
    "valid_time": "time",
    "latitude": "lat",
    "longitude": "lon",
    "pressure_level": "level",
    "u": "u_component_of_wind",
    "v": "v_component_of_wind",
    "t": "temperature",
    "q": "specific_humidity",
    "w": "vertical_velocity",
    "tisr": "toa_incident_solar_radiation",
    "z": "geopotential",
}

merged_ds = merged_ds.rename(rename_dict)

if 'number' in merged_ds:
    merged_ds = merged_ds.drop_vars('number')

merged_ds.to_netcdf(output_file)
print(f"Merged dataset saved to: {output_file}")
